In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import datetime
import time

In [2]:
import matplotlib
myfont = matplotlib.font_manager.FontProperties(fname=r'C:/Windows/Fonts/msyh.ttf')
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']   
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False

C:\Program Files\Anaconda3\lib\site-packages\matplotlib\__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
#读取user_pay_count
user_pay_count_df = pd.read_csv("./data/dataset/dataset/user_pay_count.txt", sep='\t', encoding='UTF-8', parse_dates=True, index_col=0)
user_pay_count_df.columns = user_pay_count_df.columns.map(lambda str_date:datetime.datetime.strptime(str_date, "%Y-%m-%d %H:%M:%S"))

In [4]:
#读取开店时间数据
shop_open_date = pd.read_csv("./data/dataset/dataset/shop_open_date.txt", sep='\t', index_col=0,encoding='UTF-8',parse_dates=[1])

In [5]:
###读取商家信息
shop_infos = pd.read_csv("./data/dataset/dataset/shop_info.txt",
                         encoding='UTF-8',header=None,
                        names=['id','city','loc','per_pay','score',
                               'comment_cnt','shop_level','cate_1_name','cate_2_name','cate_3_name'], index_col=0, converters={'cate_3_name':np.str})

#dtype={'id':np.int,'city':np.str,'loc':np.int,'per_pay':np.int,'score':np.int,
#                               'comment_cnt':np.int,'shop_level':np.int,'cate_1_name':np.str,'cate_2_name':np.str,'cate_3_name':np.str}

##经查看，只有cate_1_name 是超市便利店的项 cate_3_name 为NULL score为NULL comment_cnt 为NULL 且shop_level <= 1
shop_infos['cate_3_name'] = shop_infos['cate_3_name'].astype(str)

In [6]:
##读取城市名称对照表
city_names = pd.read_csv("./data/dataset/dataset/city_name.txt", index_col=0, encoding='UTF-8',parse_dates=True)

In [7]:
#读取休假表
calendar = pd.read_csv('./data/dataset/dataset/calendar.txt', sep='\t', encoding='UTF-8', index_col=0, parse_dates=True)
calendar.index = calendar.index.format()

In [8]:
#读取天气信息
def readWeatherCsv(city):
    city_pinyin = city_names.loc[city]['pinyin']
    weather_info = pd.read_csv("./data/dataset/dataset/" + city_pinyin, index_col=0, encoding='UTF-8',parse_dates=True)
    return weather_info;

In [9]:
#读取浏览信息
user_view_count_df = pd.read_csv("./data/dataset/dataset/user_view_count.txt", sep='\t', encoding='UTF-8', parse_dates=True, index_col=0)
user_view_count_df.columns = user_view_count_df.columns.map(lambda str_date:datetime.datetime.strptime(str_date, "%Y-%m-%d %H:%M:%S"))
user_view_count_df

,2015-06-25 00:00:00,2015-06-25 01:00:00,2015-06-25 02:00:00,2015-06-25 03:00:00,2015-06-25 04:00:00,2015-06-25 05:00:00,2015-06-25 06:00:00,2015-06-25 07:00:00,2015-06-25 08:00:00,2015-06-25 09:00:00,...,2016-10-31 15:00:00,2016-10-31 16:00:00,2016-10-31 17:00:00,2016-10-31 18:00:00,2016-10-31 19:00:00,2016-10-31 20:00:00,2016-10-31 21:00:00,2016-10-31 22:00:00,2016-10-31 23:00:00,2016-11-01 00:00:00
shop_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,4,0,0,2,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,0,1,0,0,3,3,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,3,2,3,4,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,1,0,0
9,0,0,0,0,0,0,0,0,0,0,...,1,0,0,2,0,0,0,0,0,0


In [10]:
def getShopPayTimeSeries(shop_id, date_range):
     return user_pay_count_df.ix[shop_id][date_range[0]:(date_range[1] + datetime.timedelta(hours=23))]

In [11]:
def getShopPayTimePeriods(shop_id, date_range, time_range):
    ts = getShopPayTimeSeries(shop_id, date_range)
    period_start = pd.date_range(start=date_range[0], end=date_range[1],freq='D')
    period_end = pd.date_range(start=date_range[0], end=date_range[1],freq='D')
    period_start = period_start + time_range[0];
    period_end = period_end + time_range[1];
    #range(len(period_start))
    #period_start[i]:period_end[i]
    
    periods = None
    for i in range(len(period_start)):
        if i == 0: 
            periods = ts[period_start[i]:period_end[i]]
        else:
            print('%d a'%(i))
            periods = periods.append(ts[period_start[i]:period_end[i]])
            print(periods)
    return periods;

In [12]:
def getShopViewTimeSeries(shop_id, date_range):
     return user_view_count_df.ix[shop_id][date_range[0]:(date_range[1] + datetime.timedelta(hours=23))]

In [13]:
def countShopViewTimePeriods(shop_id, date_range, time_range):
    ts = getShopViewTimeSeries(shop_id, date_range)
    period_start = pd.date_range(start=date_range[0], end=date_range[1],freq='D')
    period_end = pd.date_range(start=date_range[0], end=date_range[1],freq='D')
    period_start = period_start + time_range[0];
    period_end = period_end + time_range[1];
    periods = pd.Series();
    for i in range(len(period_start)):
        count = ts[period_start[i]:period_end[i]].sum()
        periods[period_start[i]] = count;
        #print('%d %s %d a'%(i, period_start[i], count))
    return periods;

In [14]:
def countShopPayTimePeriods(shop_id, date_range, time_range):
    ts = getShopPayTimeSeries(shop_id, date_range)
    period_start = pd.date_range(start=date_range[0], end=date_range[1],freq='D')
    period_end = pd.date_range(start=date_range[0], end=date_range[1],freq='D')
    period_start = period_start + time_range[0];
    period_end = period_end + time_range[1];
    #range(len(period_start))
    #period_start[i]:period_end[i]
    periods = pd.Series();
    for i in range(len(period_start)):
        count = ts[period_start[i]:period_end[i]].sum()
        periods[period_start[i]] = count;
        #print('%d %s %d a'%(i, period_start[i], count))
    return periods;

In [15]:
def plotShopPayCounts(shop_id, date_range, time_range):
    counts = countShopPayTimePeriods(shop_id, date_range, time_range)
    
    f, (ax) = plt.subplots(1, 1, figsize=(10, 5))
    
    ax.plot(counts.index, counts.values)
    info = shop_infos.ix[shop_id]
    print(type(info.cate_3_name))
    plt.title(info.cate_1_name + ' ' + info.cate_2_name + ('' if info.cate_3_name is None else info.cate_3_name), fontproperties=myfont)
    mean_value = np.mean(counts)
    ax.axhline(y=mean_value, linewidth=1, color='r')
    holiday_counts = choiceCalendarDay(counts, 0)
    #calendar.ix[s.index.strftime('%Y-%m-%d')][1]
    #calendar_filter = calendar.ix[counts.index.strftime('%Y-%m-%d')][1] == 1]
    #print(holiday_counts)
    ax.scatter(holiday_counts.index, holiday_counts.values, c='r')
    #ax.plot(holiday_counts.index, holiday_counts.values, 'r')
    plt.show()
    

In [16]:
#筛选出休息日或者工作日
def choiceCalendarDay(df, t):
    df.index = df.index.strftime('%Y-%m-%d')
    temp = calendar[calendar[1] == t]
    intersection = list(set(df.index.values) & set(temp.index.values))
    result = df[intersection];
    #print(result)
    result = pd.DataFrame(result)
    result.set_index(pd.to_datetime(result.index))
    result=result.sort_index()
    return result

In [17]:
def plotShopPayDayInfos(shop_id, start, count):
    
    info = shop_infos.ix[shop_id]
    
    f, (ax) = plt.subplots(1, 1, figsize=(15, 5))
    plt.title(info.cate_1_name + ' ' + info.cate_2_name + ('' if info.cate_3_name is None else info.cate_3_name), fontproperties=myfont)
    
    for i in range(count):
        start_time = start
        end_time = start_time + datetime.timedelta(1)
        s = getShopPayTimeSeries(id, date_range=[start_time, end_time])
        ax.plot(s.index, s.values, label=('holiday' if calendar.loc[start_time.strftime('%Y-%m-%d')][1] == 0 else 'workday'))
        start = end_time

    ax.legend()
    plt.show()

In [18]:
# date（index） | holiday | rain | count | from_open_date_day_count 
#时间维度：月份、节假日明细，周几、
#天气维度：气温、天气、风力
#历史维度：近（3,7,13,21）天均值、方差、最大值、最小值
#口碑维度：浏览量

In [19]:
# id = 4
# open_time = shop_open_date.loc[id]['date'];
# print(open_time)
# start_time = open_time;
# end_time = pd.to_datetime('2016-10-16')
# series = countShopPayTimePeriods(id, date_range=[start_time, end_time], time_range=[datetime.timedelta(hours=0), datetime.timedelta(hours=23)])
# df = series.to_frame()
#df=df.rename(columns = {0:'count'})
#df['holiday'] = calendar.loc[df.index.strftime('%Y-%m-%d')]['daytype'].values
#count_max = df['count'].quantile(0.95)
#count_min = df['count'].quantile(0.05)
#valid_df = df[(df['count'] <= count_max) & (df['count'] >= count_min)]
#temp_df = valid_df.reset_index(drop=True)

In [20]:
#获取（1、3、7）天的（均值avg 、标准差std、最大值max、最小值min）

In [21]:
def testStdCount(df, last):
    m = df.shape[0]
    if m < 2:
        return 0;
    elif m < last:
        return df.loc[0:m]['count'].std();
    else:
        return df.iloc[-last:]['count'].std();

In [22]:
def testExtremeCount(df, last):
    m = df.shape[0]
    if m == 0:
        return 0;
    elif m < last:
        return df.loc[0:m]['count'].max(), df.loc[0:m]['count'].min();
    else:
        return df.iloc[-last:]['count'].max(), df.loc[-last:]['count'].min();

In [23]:
def average_count(df, last):
    column_name = 'avg_count' + str(last);
    average_df = pd.DataFrame();
    average_df[column_name] = 0;
    data = []
    for i in range(df.shape[0]):
        if i == 0:
            data.append(df.loc[0:i]['count'].sum())
        elif i < last:
            data.append(df.loc[0:(i - 1)]['count'].sum() / i)
        else:
            data.append(df.loc[(i-last):(i - 1)]['count'].sum() / last)
            
    average_df[column_name] = data;
    return average_df;
 

In [24]:
def extreme_count(df, last):
    max_column_name = 'max_count' + str(last);
    min_column_name = 'min_count' + str(last);
    
    extreme_df = pd.DataFrame();
    extreme_df[max_column_name] = 0;
    extreme_df[min_column_name] = 0;
    
    max_data = []
    min_data = []
    
    for i in range(df.shape[0]):
        if i == 0:
            max_data.append(df.loc[0]['count']);
            min_data.append(df.loc[0]['count']);
        else:
            max_data.append(df.loc[(i-last):(i - 1)]['count'].max())
            min_data.append(df.loc[(i-last):(i - 1)]['count'].min())
            
    extreme_df[max_column_name] = max_data;
    extreme_df[min_column_name] = min_data;
    return extreme_df;

In [25]:
def std_count(df, last):
    column_name = 'std_count' + str(last);
    
    std_df = pd.DataFrame();
    std_df[column_name] = 0;
    
    data = []
    for i in range(df.shape[0]):
        if i < 2:
            data.append(0);
        else:
            data.append(df.loc[(i-last):(i - 1)]['count'].std())
    
    std_df[column_name] = data
    return std_df;

In [26]:
#df['count'][0:3].sum() / 3
#temp_df = pd.concat([average_count(temp_df, 1), temp_df], axis=1)
#temp_df = pd.concat([average_count(temp_df, 3), temp_df], axis=1)
#temp_df = pd.concat([average_count(temp_df, 7), temp_df], axis=1)
#average_count(temp_df, 3)
#average_count(temp_df, 7)
#temp_df = pd.concat([std_count(temp_df, 3), temp_df], axis=1)
#temp_df = pd.concat([std_count(temp_df, 7), temp_df], axis=1)
#temp_df = pd.concat([std_count(temp_df, 11), temp_df], axis=1)
#temp_df = pd.concat([extreme_count(temp_df, 3), temp_df], axis=1)
#result_df = temp_df.set_index(valid_df.index)
#result_df

In [27]:
# id = 8
# open_time = shop_open_date.loc[id]['date'];
# view_series = countShopViewTimePeriods(id, date_range=[open_time, pd.to_datetime('2016-10-17')], time_range=[datetime.timedelta(hours=0), datetime.timedelta(hours=23)])
# view_df = view_series.to_frame()
# view_df

In [28]:
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

In [29]:
# offset = pd.to_datetime('2016-10-17')
# X_train = result_df[:offset].drop('count',axis=1)
# y_train = result_df[:offset]['count']
# X_test = result_df[offset:].drop('count',axis=1)
# y_test = result_df[offset:]['count']

In [30]:
def evaluation(y_test, y_pred):
    n = y_test.shape[0]
    return np.sum(np.abs(y_test - y_pred) / (y_test + y_pred)) / n

In [31]:
# params = {'n_estimators': 500, 'max_depth': 2, 'min_samples_split': 3,
#           'learning_rate': 0.02, 'loss': 'ls'}
# clf = ensemble.GradientBoostingRegressor(**params) 
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)
# print("MSE: %.4f" % mse)
# evaluation(y_test, y_pred)


In [32]:
def getTestCount(id):
    series = countShopPayTimePeriods(id, date_range=[pd.to_datetime('2016-10-18'), pd.to_datetime('2016-10-31')], 
                                     time_range=[datetime.timedelta(hours=0), datetime.timedelta(hours=23)])
    df = series.to_frame()
    df=df.rename(columns = {0:'count'})
    return df['count'];
    

In [33]:
city_names = pd.read_csv("./data/dataset/dataset/city_name.txt", index_col=0, encoding='UTF-8',parse_dates=True)

In [34]:
def readWeatherData(city):
    #print(city)
    weather_info = pd.read_csv("./data/dataset/dataset/weather/process_" + city['pinyin'], 
#                               names=['max','min','desc','wind_level'],
                               index_col=0, encoding='UTF-8',parse_dates=True, dtype={'desc':object, 'wind_level':object})
    return weather_info

In [36]:
def predictNextDay(X_test):
    return clf.predict(X_test)

In [37]:
def testAverageCount(df, last):
    m = df.shape[0]
    if m == 0:
        return 0;
    elif m == 1:
        return df.loc[0]['count'];
    elif m < last:
        return df.loc[0:m]['count'].sum() / m;
    else:
        arg = df.iloc[-last:]['count'].sum() / last;
        return arg

In [38]:
def initNextDayData(date, train_df, id):
    
    avg_count1 = testAverageCount(train_df, 1)
    avg_count3 = testAverageCount(train_df, 3)
    avg_count7 = testAverageCount(train_df, 7)
    
    max_count, min_count = testExtremeCount(train_df, 3)
    
    std_count3 = testStdCount(train_df, 3)
    std_count7 = testStdCount(train_df, 7)
    std_count11 = testStdCount(train_df, 11)
    
    holiday = calendar.loc[date.strftime('%Y-%m-%d')]['daytype']
    
    #获取商店信息
    info = shop_infos.ix[id]
    #获取天气
    weather = readWeatherData(city_names.loc[info['city']])
    #获取浏览信息
    
    
    series = pd.Series([avg_count1, avg_count3, avg_count7, max_count, min_count, std_count3, std_count7, std_count11, holiday],
              index=['avg_count1','avg_count3','avg_count7','max_count3','min_count3','std_count3','std_count7','std_count11', 'holiday']);
    
    series = series.append(weather.ix[date])
    #print(series)
    return series;

In [39]:
def initTrainData(id):
    open_time = shop_open_date.loc[id]['date'];
    if open_time < pd.to_datetime('2015-7-1'):
        open_time = pd.to_datetime('2015-7-1')
    series = countShopPayTimePeriods(id, date_range=[open_time, pd.to_datetime('2016-10-17')], time_range=[datetime.timedelta(hours=0), datetime.timedelta(hours=23)])
    df = series.to_frame()
    df = df.rename(columns = {0:'count'})
    df['holiday'] = calendar.loc[df.index.strftime('%Y-%m-%d')]['daytype'].values
    
    #获取商店信息
    info = shop_infos.ix[id]
    #获取天气
    weather = readWeatherData(city_names.loc[info['city']])
    #print(weather[pd.to_datetime('2016-06-22')])
    df = pd.merge(df, weather, how='left', left_index=True, right_index=True)
    #print(df)
    
    #获取浏览信息
    #view_series = countShopViewTimePeriods(id, date_range=[open_time, pd.to_datetime('2016-10-17')], time_range=[datetime.timedelta(hours=0), datetime.timedelta(hours=23)])
    #view_df = view_series.to_frame()
    #view_df = view_df.rename(columns = {0:'view'})
    #df = pd.merge(df, view_df, how='left', left_index=True, right_index=True)
    
    #print(df)
    #result = pd.merge(left, right, how='left', left_index=True, right_index=True)
    #df['temperature_max'], df['temperature_min'], df['weather_desc'], df['wind_level'] = weather.loc[df.index.strftime('%Y-%m-%d')][]
    count_max = df['count'].quantile(0.95)
    count_min = df['count'].quantile(0.05)
    valid_df = df[(df['count'] <= count_max) & (df['count'] >= count_min)]
    
    temp_df = valid_df.reset_index(drop=True)

    temp_df = pd.concat([average_count(temp_df, 1), temp_df], axis=1)
    temp_df = pd.concat([average_count(temp_df, 3), temp_df], axis=1)
    temp_df = pd.concat([average_count(temp_df, 7), temp_df], axis=1)
    temp_df = pd.concat([extreme_count(temp_df, 3), temp_df], axis=1)
    temp_df = pd.concat([std_count(temp_df, 3), temp_df], axis=1)
    temp_df = pd.concat([std_count(temp_df, 7), temp_df], axis=1)
    temp_df = pd.concat([std_count(temp_df, 11), temp_df], axis=1)

    #temp_df = temp_df.set_index(valid_df.index)
    #result_df = temp_df.set_index(valid_df.index)

    return temp_df

In [40]:
evaluations = []
predict_date = pd.date_range(start='10/18/2016', end='10/31/2016', freq='D', normalize=True)
m = shop_open_date.shape[0];

temp = 0
evaluations = {}
for id in range(1, 2001):
    train_df = initTrainData(id)  
    #print('train_df size=%d'%(train_df.shape[0]))
    print(train_df[train_df['max'].isnull()].index)
    for date in predict_date:
        X_train, y_train = train_df.drop('count',axis=1), train_df['count']
        clf = trainModel(X_train, y_train)
        
        series = initNextDayData(date, train_df, id)
        next_dataframe = series.to_frame().T
        next_dataframe['holiday'] = next_dataframe['holiday'].astype(np.int32);
        #print(next_dataframe)
        count = predictNextDay(next_dataframe)
        series['count'] = count;
        temp = series.to_frame()
        #print(temp)
        train_df = train_df.append(series, ignore_index = True)
        train_df['count'] = train_df['count'].astype(np.int32);
        train_df['holiday'] = train_df['holiday'].astype(np.int32);
        
    e = evaluation(getTestCount(id).values, train_df.iloc[-14:]['count'].values)
    print("id=%d evaluation=%f"%(id, e))
    evaluations[id] = e

Int64Index([], dtype='int64')
id=1 evaluation=0.090103
Int64Index([], dtype='int64')
id=2 evaluation=0.362629
Int64Index([], dtype='int64')
id=3 evaluation=0.118922
Int64Index([], dtype='int64')
id=4 evaluation=0.184378
Int64Index([], dtype='int64')
id=5 evaluation=0.554605
Int64Index([], dtype='int64')
id=6 evaluation=0.591527
Int64Index([], dtype='int64')
id=7 evaluation=0.484438
Int64Index([], dtype='int64')
id=8 evaluation=0.179279
Int64Index([], dtype='int64')
id=9 evaluation=0.285026
Int64Index([], dtype='int64')
id=10 evaluation=0.708371
Int64Index([], dtype='int64')
id=11 evaluation=0.616978
Int64Index([], dtype='int64')
id=12 evaluation=0.135626
Int64Index([], dtype='int64')
id=13 evaluation=0.605196
Int64Index([], dtype='int64')
id=14 evaluation=0.126396
Int64Index([], dtype='int64')
id=15 evaluation=0.383644
Int64Index([], dtype='int64')
id=16 evaluation=0.136119
Int64Index([], dtype='int64')
id=17 evaluation=0.388115
Int64Index([], dtype='int64')
id=18 evaluation=0.453498
I

In [45]:
train_df

,std_count11,std_count7,std_count3,max_count3,min_count3,avg_count7,avg_count3,avg_count1,count,holiday,max,desc,wind_level,min
0,0.000000,0.000000,0.000000,3,3,3.000000,3.000000,3.0,3,3,NaN,NaN,NaN,NaN
1,0.000000,0.000000,0.000000,3,3,3.000000,3.000000,3.0,3,3,20.0,4,1,19.0
2,0.000000,0.000000,0.000000,3,3,3.000000,3.000000,3.0,3,5,22.0,3,1,18.0
3,0.000000,0.000000,0.000000,3,3,3.000000,3.000000,3.0,6,1,22.0,2,1,18.0
4,1.500000,1.500000,1.732051,6,3,3.750000,4.000000,6.0,3,1,24.0,3,1,21.0
5,1.341641,1.341641,1.732051,6,3,3.600000,4.000000,3.0,3,1,29.0,4,1,23.0
6,1.224745,1.224745,1.732051,6,3,3.500000,4.000000,3.0,4,2,26.0,4,1,24.0
7,1.133893,1.133893,0.577350,4,3,3.571429,3.333333,4.0,4,3,32.0,1,1,24.0
8,1.060660,1.112697,0.577350,4,3,3.714286,3.666667,4.0,4,5,35.0,1,1,25.0
9,1.000000,1.069045,0.000000,4,4,3.857143,4.000000,4.0,13,1,35.0,1,1,25.0


In [1]:
predict_date = pd.date_range(start='10/18/2016', end='10/31/2016', freq='D', normalize=True)
m = shop_open_date.shape[0];

temp = 0
id = 742
train_df = initTrainData(id)  
#print('train_df size=%d'%(train_df.shape[0]))
print(train_df[train_df['max'].isnull()].index)
for date in predict_date:
    X_train, y_train = train_df.drop('count',axis=1), train_df['count']
    clf = trainModel(X_train, y_train)

    series = initNextDayData(date, train_df, id)
    next_dataframe = series.to_frame().T
    next_dataframe['holiday'] = next_dataframe['holiday'].astype(np.int32);
    #print(next_dataframe)
    count = predictNextDay(next_dataframe)
    series['count'] = count;
    temp = series.to_frame()
    #print(temp)
    train_df = train_df.append(series, ignore_index = True)
    train_df['count'] = train_df['count'].astype(np.int32);
    train_df['holiday'] = train_df['holiday'].astype(np.int32);

y_Test = getTestCount(id).values
y_Pred = train_df.iloc[-14:]['count'].values
e = evaluation(y_Test, y_Pred)
print("id=%d evaluation=%f"%(id, e))
print(y_Test)
print(y_Pred)

NameError: name 'pd' is not defined

In [102]:
def trainModel(X_train, y_train):
    params = {'n_estimators': 400, 'max_depth': 2, 'min_samples_split': 3,
          'learning_rate': 0.01, 'loss': 'ls'}
    clf = ensemble.GradientBoostingRegressor(**params) 
    clf.fit(X_train, y_train)
    return clf;

In [41]:
evaluation_df = pd.DataFrame.from_dict(evaluations, orient='index');
evaluation_df.columns = ['evaluation']
evaluation_df.index.name = 'id'
evaluation_df.sort_values('evaluation')
evaluation_df

,evaluation
id,
1,0.090103
2,0.362629
3,0.118922
4,0.184378
5,0.554605
6,0.591527
7,0.484438
8,0.179279
9,0.285026


In [42]:
evaluation_df.to_csv("./data/dataset/dataset/evaluation.txt", sep='\t', index=True, encoding='UTF-8')

In [77]:
good_pred = evaluation_df[evaluation_df['evaluation'] <= 0.15]

In [78]:
good_pred.shape

(432, 1)

In [46]:
train_df.iloc[-2:]['count'].sum()

615

In [67]:
train_df.iloc[86]['count'].sum()

-1.0

In [2]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
                    'B': ['B0', 'B1', 'B2'],
                    'C': ['C0', 'C1', 'C2']},
                    index=pd.Index(['K0', 'K1', 'K2'], name='key')) 
 

right = pd.DataFrame({'C': ['C0', 'C1', 'C2','C3'],
                       'D': ['D0', 'D1', 'D2','D3']},
                       index=pd.Index(['K0', 'K1', 'K2', 'K3'], name='key'))
 

result = pd.merge(left, right, how='left', left_index=True, right_index=True)

In [3]:
result

,A,B,C_x,C_y,D
key,,,,,
K0,A0,B0,C0,C0,D0
K1,A1,B1,C1,C1,D1
K2,A2,B2,C2,C2,D2


In [45]:
city_names.loc['安康']

pinyin    ankang
Name: 安康, dtype: object